In [43]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.mllib.classification import SVMWithSGD, SVMModel
import numpy as np
import pandas as pd
import os

In [44]:
# spark = SparkSession.builder.master("local").appName("NaiveBayesJob").getOrCreate()
# conf = SparkConf().setMaster("local").setAppName("HousingJobSparkShort")
# sc = SparkContext(conf=conf)

conf = SparkConf().setMaster("local").setAppName("TestJobSpark")
sc = SparkContext(conf=conf)


In [90]:
def load_data(file_path, file_name):
    full_path = os.path.join(file_path, file_name)
    return pd.read_csv(full_path)

def mapOceanProximity(proximity):

    if(proximity == "NEAR BAY"):
        return 1
    
    elif(proximity == "INLAND"):
        return 0
    
    # elif(proximity =="<1H OCEAN"):
    #     return 0
    
    # elif(proximity=="NEAR OCEAN"):
    #     return 1
    
    # elif(proximity =="ISLAND"):
    #     return 1
    
    # else:
    #     return 0

def createLabeledPoint(fields):

    longitude = float(fields[0])
    latitude = float(fields[1])
    housing_median_age = float(fields[2])
    total_rooms = float(fields[3])
    total_bedrooms = float(fields[4])
    population = float(fields[4])
    households = float(fields[5])
    median_income = float(fields[6])
    median_house_value = float(fields[7])
    ocean_proximity = mapOceanProximity(fields[8])

    return LabeledPoint(ocean_proximity, np.array([longitude, latitude, housing_median_age, total_rooms,
                        total_bedrooms, population, households, median_income, median_house_value]))

In [91]:
DATA_PATH = os.path.join(os.getcwd(),'datasets', 'housing')
FILE_NAME = "housing_short.csv"
housing_data = load_data(DATA_PATH, FILE_NAME)

data_rdd = sc.textFile(os.path.join(DATA_PATH, FILE_NAME))

header = data_rdd.first()
data_rdd= data_rdd.filter(lambda x:x != header) 
data_rdd = data_rdd.map(lambda line: line.split(","))
training_data_rdd, test_data_rdd = data_rdd.randomSplit(weights=[8.0,2.0], seed=1)  # raw_data_rdd.map(createLabeledPoint)
training_data_rdd, test_data_rdd = training_data_rdd.map(createLabeledPoint), test_data_rdd.map(createLabeledPoint)


In [80]:
# for rows in data_rdd.collect():
#     print(rows)



In [75]:
# data_rdd.na.drop(subset=["total_bedrooms", "longitude","latitude", "housing_median_age", "total_rooms", "population", "households", "median_income", "median_house_value", "ocean_proximity"])
# training_data_rdd, test_data_rdd = data_rdd.randomSplit(weights=[8.0,2.0], seed=1)  # raw_data_rdd.map(createLabeledPoint)


In [40]:
header = housing_rdd.first()
housing_rdd  = housing_rdd.filter(lambda x: x != header)
# housing_rdd = housing_rdd.map(lambda x: x.split(","))

# training_data_rdd, test_data_rdd = housing_rdd.randomSplit(weights=[8.0,2.0], seed=1)  # raw_data_rdd.map(createLabeledPoint)

NameError: name 'housing_rdd' is not defined

In [32]:
train_rdd, test_rdd = training_data_rdd.map(createLabeledPoint), test_data_rdd.map(createLabeledPoint)


In [34]:
# model = SVMWithSGD.train(train_rdd, iterations=100)

train_rdd.collect()


21/07/02 19:58:17 ERROR Executor: Exception in task 0.0 in stage 9.0 (TID 9)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/zeefu/anaconda3/envs/pyspark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/home/zeefu/anaconda3/envs/pyspark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/zeefu/anaconda3/envs/pyspark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/zeefu/anaconda3/envs/pyspark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_32561/2965744094.py", line 27, in createLabeledPoint
ValueError: could not convert string to float: '-'

	

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 9) (192.168.0.155 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/zeefu/anaconda3/envs/pyspark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/home/zeefu/anaconda3/envs/pyspark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/zeefu/anaconda3/envs/pyspark/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/zeefu/anaconda3/envs/pyspark/lib/python3.8/site-packages/pyspark